In [1]:
import torch
print(torch.__version__)

# 2.3.0+cu121 인데, 업그레이드 하기
!pip install -q torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu118

2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.7/839.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/FMA-Net_inside_test')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/FMA-Net_inside_test

/content/drive/MyDrive/FMA-Net_inside_test


In [4]:
import os
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [5]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 726.7 kB/s eta 0:00:00


In [6]:
torch.cuda.empty_cache()

In [ ]:
!rm -r /content/drive/MyDrive/brids3/birds1_resized


In [ ]:
!ffmpeg -i /content/drive/MyDrive/0704/2/2.mov -q:v 0 /content/drive/MyDrive/0704/2/2.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [9]:
import cv2
import os
import math
from PIL import Image

def get_video_fps(video_path):
    # 동영상 파일 열기
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print("동영상을 열 수 없습니다.")
        return None

    # FPS 값 가져오기
    fps = video.get(cv2.CAP_PROP_FPS)

    # 동영상 파일 닫기
    video.release()

    return fps


def extract_frames(video_path, output_folder, target_fps=60):
    # 동영상 파일 열기
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print("동영상을 열 수 없습니다.")
        return

    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 동영상의 원래 FPS 가져오기
    original_fps = round(video.get(cv2.CAP_PROP_FPS))

    # 프레임 간격 계산 (원본 FPS와 목표 FPS 비율에 따라)
    frame_interval = int(original_fps / target_fps)

    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idx = 0

    while True:
        ret, frame = video.read()

        if not ret:
            break

        if frame_idx % frame_interval == 0:
            frame_number = int(frame_idx / frame_interval)
            frame_filename = os.path.join(output_folder, f"frame_{frame_number:04d}.png")
            cv2.imwrite(frame_filename, frame)

        frame_idx += 1

    # 동영상 파일 닫기
    video.release()
    print(f"총 {frame_number + 1}개의 프레임이 저장되었습니다.")


# 몇 fps인지 확인할 동영상 파일 경로 ====> 영상 이름 입력 필요
video_path = '/content/drive/MyDrive/0704/3_cut_2x/buyang_bird_0704_cut_2x.mp4'

fps = get_video_fps(video_path)
if fps is not None:
    print(f"동영상의 FPS: {fps}")

# 이미지 프레임으로 변환할 동영상 파일 경로와 출력 폴더 경로 ===> 폴더 이름 입력 필요
folder_path = '/content/drive/MyDrive/FMA-Net_inside_test/test_blur/3_bird_cut_2x'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"폴더가 생성되었습니다: {folder_path}")

# 프레임 추출 함수 실행
extract_frames(video_path, folder_path)

############
### 이미지 리사이즈

# 원본 이미지가 저장된 디렉토리와 리사이즈된 이미지가 저장될 디렉토리 경로
input_dir = folder_path  # 원본 이미지 디렉토리

# ===> 폴더 이름 입력 필요
#output_dir = '/content/drive/MyDrive/brids3/birds1_resized'  # 리사이즈된 이미지 디렉토리
output_dir = '/content/drive/MyDrive/FMA-Net_inside_test/test_blur/3_bird_resized_cut_2x'  # 리사이즈된 이미지 디렉토리

# 출력 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 첫 번째 이미지의 높이와 너비 가져오기
first_image_file = os.listdir(input_dir)[0]
first_image_path = os.path.join(input_dir, first_image_file)
first_image = Image.open(first_image_path)
original_width, original_height = first_image.size

# 리사이즈할 비율
resize_ratio = 0.25

# 새로운 해상도 계산
new_width = int(original_width * resize_ratio)
new_height = int(original_height * resize_ratio)

# 이미지 파일 목록 가져오기
image_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# 모든 이미지에 대해 리사이즈 작업 수행
for i, image_file in enumerate(image_files):
    # 이미지 열기
    img_path = os.path.join(input_dir, image_file)
    img = Image.open(img_path)

    # 이미지 리사이즈
    img_resized = img.resize((new_width, new_height), Image.LANCZOS)

    # 리사이즈된 이미지 저장
    output_path = os.path.join(output_dir, image_file)
    img_resized.save(output_path)

    print(f"Processed {i + 1}/{len(image_files)}: {image_file}")

print("이미지 리사이즈 완료!")


동영상의 FPS: 60.0
폴더가 생성되었습니다: /content/drive/MyDrive/FMA-Net_inside_test/test_blur/3_bird_cut_2x
총 214개의 프레임이 저장되었습니다.
Processed 1/214: frame_0000.png
Processed 2/214: frame_0001.png
Processed 3/214: frame_0002.png
Processed 4/214: frame_0003.png
Processed 5/214: frame_0004.png
Processed 6/214: frame_0005.png
Processed 7/214: frame_0006.png
Processed 8/214: frame_0007.png
Processed 9/214: frame_0008.png
Processed 10/214: frame_0009.png
Processed 11/214: frame_0010.png
Processed 12/214: frame_0011.png
Processed 13/214: frame_0012.png
Processed 14/214: frame_0013.png
Processed 15/214: frame_0014.png
Processed 16/214: frame_0015.png
Processed 17/214: frame_0016.png
Processed 18/214: frame_0017.png
Processed 19/214: frame_0018.png
Processed 20/214: frame_0019.png
Processed 21/214: frame_0020.png
Processed 22/214: frame_0021.png
Processed 23/214: frame_0022.png
Processed 24/214: frame_0023.png
Processed 25/214: frame_0024.png
Processed 26/214: frame_0025.png
Processed 27/214: frame_0026.png
P

In [ ]:
%rm

rm: missing operand
Try 'rm --help' for more information.


In [10]:
# test on your own datasets
torch.backends.cudnn.enabled = False

%cd /content/drive/MyDrive/FMA-Net_inside_test
!python main.py --test_custom --config_path experiment.cfg

/content/drive/MyDrive/FMA-Net_inside_test
num custom dataset: 212
load degradation learning network status from ./results/FMA-Net/model_stage2/model_D_best.pt, epoch: 400
load restoration network status from ./results/FMA-Net/model_stage2/model_R_best.pt, epoch: 400


In [11]:
!pip install opencv-python


import cv2
import os

def create_video_from_frames(frames_folder, output_file, fps=30):
    # Get the list of all frame files in the directory
    frames = [f for f in os.listdir(frames_folder) if os.path.isfile(os.path.join(frames_folder, f))]
    frames.sort()  # Make sure frames are in correct order
    # frames = frames[:90] # 90 frame으로만 영상 만들고 싶을 때 사용

    # Read the first frame to get the frame size
    first_frame = cv2.imread(os.path.join(frames_folder, frames[0]))
    height, width, layers = first_frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    for frame in frames:
        img = cv2.imread(os.path.join(frames_folder, frame))
        video.write(img)

    video.release()

In [14]:
# 결과로 나온 이미지 프레임 시퀀스 --> 영상으로 변환
frames_folder = "/content/drive/MyDrive/FMA-Net_inside_test/results/FMA-Net/test/3_bird_resized_cut_2x"  # Replace with your frames folder path
output_file = '/content/drive/MyDrive/0704/3_cut_2x/3_done_deblur_new.mp4'  # Replace with your desired output file name

create_video_from_frames(frames_folder, output_file, fps=60)

In [15]:
# resized 한 영상 만들기 (고양이)
frames_folder = '/content/drive/MyDrive/FMA-Net_inside_test/test_blur/3_bird_resized_cut_2x'  # Replace with your frames folder path
output_file = '/content/drive/MyDrive/0704/3_cut_2x/3_downsampling.mp4'  # Replace with your desired output file name

create_video_from_frames(frames_folder, output_file, fps=60)

In [ ]:
# FMA직전 영상 생성 (원본을 1/4로 다운샘플링만 한 영상 - 비교용으로)
frames_folder = '/content/drive/MyDrive/brids3/birds1_resized'  # Replace with your frames folder path
output_file = '/content/drive/MyDrive/birds1_src.mp4'  # Replace with your desired output file name

create_video_from_frames(frames_folder, output_file, fps=30)
